In [15]:
import pyomo.environ as pe
import numpy as np
import pandas as pd

In [5]:
m = pe.ConcreteModel()
m.a = pe.Set(initialize=[1, 2, 3])
m.x = pe.Var(m.a, initialize=0, bounds=(-10,10))
m.y = pe.Var(m.a)
def c_rule(m, i):
    return m.x[i] >= m.y[i]
m.c = pe.Constraint(m.a, rule=c_rule)
m.c.pprint()
m.z = pe.Var()
m.c2 = pe.Constraint(pe.Any)
m.c2[1] = m.x[1] == 5 * m.z
m.c2[8] = m.x[2] == m.z * m.y[2]
m.c2.pprint()

c : Size=3, Index=a, Active=True
    Key : Lower : Body        : Upper : Active
      1 :  -Inf : y[1] - x[1] :   0.0 :   True
      2 :  -Inf : y[2] - x[2] :   0.0 :   True
      3 :  -Inf : y[3] - x[3] :   0.0 :   True
c2 : Size=2, Index=Any, Active=True
    Key : Lower : Body            : Upper : Active
      1 :   0.0 :      x[1] - 5*z :   0.0 :   True
      8 :   0.0 : x[2] - z * y[2] :   0.0 :   True


In [7]:
# Initialize sets and components
m.A = pe.Set(initialize=[2,3,5])
m.B = pe.Set(initialize=set([2,3,5]))
m.C = pe.Set(initialize=(2,3,5))

In [8]:
m.A.pprint()

A : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=(2, 5)
    [2, 3, 5]


In [11]:
# using generators for inititalization
m.D = pe.Set(initialize=range(9))
m.E = pe.Set(initialize=(i for i in m.B if i%2 == 0))

    'pyomo.core.base.sets.SimpleSet'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.sets.SimpleSet'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [12]:
m.E.pprint()

E : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=(2, 2)
    [2]


In [17]:
# using numpy
f = np.array([2, 3, 5])
m.F = pe.Set(initialize=f)

In [18]:
# using functions for initialization
def g(model):
    return [2,3,5]
m.G = pe.Set(initialize=g)

In [19]:
m.G.pprint()

G : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=(2, 5)
    [2, 3, 5]


In [21]:
# using indexed sets

H_init = {}
H_init[2] = [1,3,5]
H_init[3] = [2,4,6]
H_init[4] = [3,5,7]
m.H = pe.Set([2,3,4],initialize=H_init)

In [23]:
# parameters
m.a = pe.Param(initialize=1.1)
m.b = pe.Param([1,2,3], initialize={1:1, 2:2, 3:3})
def c(model):
    return {1:1, 2:2, 3:3}
m.c = pe.Param([1,2,3], initialize=c)

    'pyomo.core.base.sets.SimpleSet'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.param.SimpleParam'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


The following commands are used to declare data:
- The set command declares set data.
- The param command declares a table of parameter data, which can also include the declaration of the set data used to index the parameter data.
- The table command declares a two-dimensional table of parameter data.
- The load command defines how set and parameter data is loaded from external data sources, including ASCII table files, CSV files, XML files, YAML files, JSON files, ranges in spreadsheets, and database tables.

The following commands are also used in data command files:
- The include command specifies a data command file that is processed immediately.
- The data and end commands do not perform any actions, but they provide compatibility with AMPL scripts that define data commands.
- The namespace keyword allows data commands to be organized into named groups that can be enabled or disabled during model construction.

The following data types can be represented in a data command file:
- Numeric value: Any Python numeric value (e.g. integer, float, scientific notation, or boolean).
- Simple string: A sequence of alpha-numeric characters.
- Quoted string: A simple string that is included in a pair of single or double quotes. A quoted string can include quotes within the quoted string

In [1]:
# sodacan-pyomo.py

from pyomo.environ import *
from math import pi

m = ConcreteModel()

m.r = Var(bounds=(0,None))
m.h = Var(bounds=(0,None))

m.o = Objective(expr=2*pi*m.r*(m.r + m.h))
m.c = Constraint(expr=pi*m.h*m.r**2 == 355)

solver = SolverFactory('ipopt')
status = solver.solve(m)

print("Status = %s" % status.solver.termination_condition)

print("%s = %f" % (m.r, value(m.r)))
print("%s = %f" % (m.h, value(m.h)))
print("Objective = %f" % value(m.o))

Status = optimal
r = 3.837215
h = 7.674430
Objective = 277.545024


## Telecom Tower Problem

In [39]:
from pyomo.environ import *
from math import pi

m = ConcreteModel()

m.x = Var(initialize=1e-10)#(bounds=(1e-10, None))
m.y = Var(initialize=1e-10)#(bounds=(1e-10, None))

m.a = Param([1,2,3,4],  initialize={1:0, 2:-110, 3:-130, 4:-250})
m.b = Param([1,2,3,4],  initialize={1:0, 2:60, 3:20, 4:100})

m.c1 = Constraint(expr= ((m.x - m.a[1])**2 + (m.y - m.b[1])**2)**0.5 <= 160)
m.c2 = Constraint(expr= ((m.x - m.a[2])**2 + (m.y - m.b[2])**2)**0.5 <= 160)
m.c3 = Constraint(expr= ((m.x - m.a[3])**2 + (m.y - m.b[3])**2)**0.5 <= 160)
m.c4 = Constraint(expr= ((m.x - m.a[4])**2 + (m.y - m.b[4])**2)**0.5 <= 160)
m.o = Objective(expr= 
                ((m.x - m.a[1])**2 + (m.y - m.b[1])**2)**0.5 + \
                ((m.x - m.a[2])**2 + (m.y - m.b[2])**2)**0.5 + \
                ((m.x - m.a[3])**2 + (m.y - m.b[3])**2)**0.5 + \
                ((m.x - m.a[4])**2 + (m.y - m.b[4])**2)**0.5 , sense=minimize)

solver = SolverFactory('ipopt')
status = solver.solve(m)

print("Status = %s" % status.solver.termination_condition)

print("%s = %f" % (m.x, value(m.x)))
print("%s = %f" % (m.y, value(m.y)))
print("Objective = %f" % value(m.o))

Status = optimal
x = -116.666667
y = 46.666667
Objective = 313.979600


In [23]:
m.a[2]

-110

In [17]:
(250**2 + 0**2)**0.5

250.0